In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))

    return epochs, pp_directory

In [3]:
def direction_index(task, direction, pp):
    if task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\aln_MovementWorkspace_direction.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_MovementWorkspace_direction.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_MovementWorkspace_direction.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_MovementWorkspace_direction.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_MovementWorkspace_direction.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if direction == v]

    numtrials = len(index)
    
    return index, numtrials

In [4]:
def error_index(task, errsize, pp):
    
    if task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_ErrorSize_index.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_ErrorSize_index.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_ErrorSize_index.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_ErrorSize_index.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if errsize == v]

    numtrials = len(index)
    
    return index, numtrials

In [5]:
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)

In [6]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
      
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]

        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [7]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, err, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_SmallLarge_%s_%s_%s-epo.fif' % (pp_num, condname, err, task))
    data.save(out_fname, overwrite = True)

In [8]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, err, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (pp_num, condname, err, task))
    evoked.save(out_fname, overwrite = True)

In [9]:
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
l_cutoff = 30

#ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        diridx, dirtrials = direction_index('aln', conditions[cond], pp)
        aldata = epoch_trials('aln', pp, data, diridx)
        output_condition_epochs(pp, aldata, ppdir, conditionnames[cond], 'SL', 'aligned')
        #filter at 30 Hz
        aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, aldata, ppdir, conditionnames[cond], 'SL', 'aligned')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available


...

[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 768 out of 768 | elapsed:    0.2s finished


In [10]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']

#aligned
for condition in range(0, len(conditionnames)):
    flist = []
    evoked_list = []
    for pp in participants:
        id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
        pp_directory = os.path.join(id_directory, erps)
        fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], 'SL', 'aligned'))
        evoked = mne.read_evokeds(fname)
        flist.append(evoked)
        evoked = flist[pp][0]
        evoked_list.append(evoked)
    if condition == 0:
        aln_right_grand = mne.grand_average(evoked_list)
        aln_right_flist = evoked_list
    elif condition == 1:
        aln_left_grand = mne.grand_average(evoked_list)
        aln_left_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_SmallLarge_rightmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_SmallLarge_rightmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_SmallLarge_rightmoves_SL_aligned-ave.fif ...
    Found the data of interest:


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_SmallLarge_rightmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_SmallLarge_rightmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12

        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_SmallLarge_leftmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_SmallLarge_leftmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the m

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_SmallLarge_leftmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_SmallLarge_leftmoves_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 12 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] 

In [11]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
errsizes = ['sml', 'lrg']
tasks = ['rot', 'mir']
l_cutoff = 30

# ROTATED AND MIRROR TRAINING

for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for size in range(0, len(errsizes)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                if len(diridx) == 0:
                    break
                erridx, errtrials = error_index(tasks[task], errsizes[size], pp)
                #print(diridx, erridx)
                idx = [v for i,v in enumerate(diridx) if v in set(erridx)]
                #print(idx)
                pdata = epoch_trials(tasks[task], pp, data, idx)
                #print(pdata)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], errsizes[size], tasks[task])
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], errsizes[size], tasks[task])

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1088 out of 1088 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1152 out of 1152 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 1280 out of 1280 | elapsed:    0.4s finished


In [12]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rot', 'mir']
errsizes = ['sml', 'lrg']
#rot and mir
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for size in range(0, len(errsizes)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], errsizes[size], tasks[task]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and size == 0:
                rot_right_sml_grand = mne.grand_average(evoked_list)
                rot_right_sml_flist = evoked_list
            elif task == 0 and condition == 0 and size == 1:
                rot_right_lrg_grand = mne.grand_average(evoked_list)
                rot_right_lrg_flist = evoked_list
            elif task == 0 and condition == 1 and size == 0:
                rot_left_sml_grand = mne.grand_average(evoked_list)
                rot_left_sml_flist = evoked_list
            elif task == 0 and condition == 1 and size == 1:
                rot_left_lrg_grand = mne.grand_average(evoked_list)
                rot_left_lrg_flist = evoked_list
            elif task == 1 and condition == 0 and size == 0:
                mir_right_sml_grand = mne.grand_average(evoked_list)
                mir_right_sml_flist = evoked_list
            elif task == 1 and condition == 0 and size == 1:
                mir_right_lrg_grand = mne.grand_average(evoked_list)
                mir_right_lrg_flist = evoked_list
            elif task == 1 and condition == 1 and size == 0:
                mir_left_sml_grand = mne.grand_average(evoked_list)
                mir_left_sml_flist = evoked_list
            elif task == 1 and condition == 1 and size == 1:
                mir_left_lrg_grand = mne.grand_average(evoked_list)
                mir_left_lrg_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_SmallLarge_rightmoves_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 17 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_SmallLarge_rightmoves_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 26 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_SmallLarge_rightmoves_sml_rot-ave.fif ...
    Found the data of interest:
        t

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 19 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_SmallLarge_rightmoves_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\lrp\p009_SmallLarge_rightmoves_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 25 - asp

        nave = 19 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\lrp\p011_SmallLarge_leftmoves_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 24 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\lrp\p016_SmallLarge_leftmoves_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 19 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\lrp\p018_SmallLarge_leftmoves_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 14 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p019\lrp\p019_SmallLarge_leftmoves_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 17 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p024\lrp\p024_SmallLarge_leftmo

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 21 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\lrp\p030_SmallLarge_rightmoves_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 20 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\lrp\p031_SmallLarge_rightmoves_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 18 - asp

        0 CTF compensation matrices available
        nave = 14 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p005\lrp\p005_SmallLarge_leftmoves_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 16 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p006\lrp\p006_SmallLarge_leftmoves_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 23 - aspect type = 100
No projector specified for this dataset. Please consider the method 

Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p012\lrp\p012_SmallLarge_leftmoves_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 18 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\lrp\p013_SmallLarge_leftmoves_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 20 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_SmallLarge_leftmo

In [13]:
#save as evoked objects
conditions = ['r', 'l']
conditionnames = ['rightmoves', 'leftmoves']
errsizes = ['sml', 'lrg']
l_cutoff = 30

#RANDOM
tasks = ['rdmrot', 'rdmmir']

#only half will have rdm in rot and other half will have rdm in mir, due to us only considering targets along horizontal axis (i.e., vertical mirror)
for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for cond in range(0, len(conditions)):
            for size in range(0, len(errsizes)):
                diridx, dirtrials = direction_index(tasks[task], conditions[cond], pp)
                if len(diridx) == 0:
                    break
                erridx, errtrials = error_index(tasks[task], errsizes[size], pp)
                #print(diridx, erridx)
                idx = [v for i,v in enumerate(diridx) if v in set(erridx)]
                #print(idx)
                triallist = trial_list_rdm(tasks[task], pp, idx)
                pdata = data[triallist]
                print(pdata)
                output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], errsizes[size], 'rdm')
                #filter at 30 Hz
                pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
                #then average across trials
                output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], errsizes[size], 'rdm')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
<EpochsFIF |  15 events (all good), -2 - 1.995 sec, baseline -1.3 – -1 sec, ~6.0 MB, data loaded,
 '16140': 15>
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


<EpochsFIF |  8 events (all good), -2 - 1.995 sec, baseline -1.3 – -1 sec, ~3.3 MB, data loaded,
 '16140': 8>
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 960 out of 960 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


<EpochsFIF |  6 events (all good), -2 - 1.995 sec, baseline -1.3 – -1 sec, ~2.5 MB, data loaded,
 '16140': 6>
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


<EpochsFIF |  16 events (all good), -2 - 1.995 sec, baseline -1.3 – -1 sec, ~6.4 MB, data loaded,
 '16140': 16>
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


<EpochsFIF |  5 events (all good), -2 - 1.995 sec, baseline -1.3 – -1 sec, ~2.1 MB, data loaded,
 '16140': 5>
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 1024 out of 1024 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:    0.0s finished


In [14]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
conditionnames = ['rightmoves', 'leftmoves']
tasks = ['rdm']
errsizes = ['sml', 'lrg']
#rdm
for task in range(0, len(tasks)):
    for condition in range(0, len(conditionnames)):
        for size in range(0, len(errsizes)):
            flist = []
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], errsizes[size], tasks[task]))
                if os.path.exists(fname) == False:
                    continue
#                 print(fname)
                evoked = mne.read_evokeds(fname)
                flist.append(evoked)
                evoked = evoked[0]
                evoked_list.append(evoked)
#                 print(evoked_list)
            if task == 0 and condition == 0 and size == 0:
                rdm_right_sml_grand = mne.grand_average(evoked_list)
                rdm_right_sml_flist = evoked_list
            elif task == 0 and condition == 0 and size == 1:
                rdm_right_lrg_grand = mne.grand_average(evoked_list)
                rdm_right_lrg_flist = evoked_list
            elif task == 0 and condition == 1 and size == 0:
                rdm_left_sml_grand = mne.grand_average(evoked_list)
                rdm_left_sml_flist = evoked_list
            elif task == 0 and condition == 1 and size == 1:
                rdm_left_lrg_grand = mne.grand_average(evoked_list)
                rdm_left_lrg_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\lrp\p000_SmallLarge_rightmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 15 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_SmallLarge_rightmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 13 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_SmallLarge_rightmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t

        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 10 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_SmallLarge_rightmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 15 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_SmallLarge_rightmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 11 - aspect type = 100
No projector spec

        nave = 11 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p007\lrp\p007_SmallLarge_rightmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 8 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\lrp\p008_SmallLarge_rightmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 11 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p026\lrp\p026_SmallLarge_rightmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 13 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\lrp\p027_SmallLarge_rightmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 9 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\lrp\p028_SmallLarge_rightmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t 

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 9 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\lrp\p014_SmallLarge_leftmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 10 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\lrp\p015_SmallLarge_leftmoves_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 8 - aspect 

        nave = 13 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\lrp\p001_SmallLarge_leftmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 13 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\lrp\p002_SmallLarge_leftmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 11 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\lrp\p020_SmallLarge_leftmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 14 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\lrp\p021_SmallLarge_leftmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16140)
        0 CTF compensation matrices available
        nave = 10 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.3, -1] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\lrp\p022_SmallLarge_leftmoves_lrg_rdm-ave.fif ...
    Found the data of interest:
        t = 

In [15]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln_right', 'aln_left',
          'rot_right_sml', 'rot_right_lrg', 'rot_left_sml', 'rot_left_lrg',
          'mir_right_sml', 'mir_right_lrg', 'mir_left_sml', 'mir_left_lrg',
          'rdm_right_sml', 'rdm_right_lrg', 'rdm_left_sml', 'rdm_left_lrg']

channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        if f == 'aln_right':
            for pp in range(0, len(aln_right_flist)):
                df = aln_right_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'aln_left':
            for pp in range(0, len(aln_left_flist)):
                df = aln_left_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_sml':
            for pp in range(0, len(rot_right_sml_flist)):
                df = rot_right_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_right_lrg':
            for pp in range(0, len(rot_right_lrg_flist)):
                df = rot_right_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_sml':
            for pp in range(0, len(rot_left_sml_flist)):
                df = rot_left_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rot_left_lrg':
            for pp in range(0, len(rot_left_lrg_flist)):
                df = rot_left_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_sml':
            for pp in range(0, len(mir_right_sml_flist)):
                df = mir_right_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_right_lrg':
            for pp in range(0, len(mir_right_lrg_flist)):
                df = mir_right_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_sml':
            for pp in range(0, len(mir_left_sml_flist)):
                df = mir_left_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'mir_left_lrg':
            for pp in range(0, len(mir_left_lrg_flist)):
                df = mir_left_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_sml':
            for pp in range(0, len(rdm_right_sml_flist)):
                df = rdm_right_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_right_lrg':
            for pp in range(0, len(rdm_right_lrg_flist)):
                df = rdm_right_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_sml':
            for pp in range(0, len(rdm_left_sml_flist)):
                df = rdm_left_sml_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
        elif f == 'rdm_left_lrg':
            for pp in range(0, len(rdm_left_lrg_flist)):
                df = rdm_left_lrg_flist[pp].to_data_frame()
                time = df['time'].tolist()
                df = df[[channel]]
                ppdat = []
                for timept in range(0,800):
                    subdf = df.loc[timept,:].mean()
                    ppdat.append(subdf)
                newdf['pp%03d'% pp] = ppdat
            newdf['time'] = time
            newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_%s.csv' % (f, channel))
            newdf.to_csv(newdf_filename)
             


In [16]:
# # Difference waves between aligned and perturbed conditions for C3
# #channels = ['C3']
# channels = ['C4']
# # channels = ['Oz','O1','O2']
# evokeds = dict(aln_right = right_aln_flist, 
#                aln_left = left_aln_flist,
#                rot_right = right_rot_flist,
#                rot_left = left_rot_flist,
#                rdm_right = right_rdm_flist,
#                rdm_left = left_rdm_flist,
#                mir_right = right_mir_flist,
#                mir_left = left_mir_flist)
# #plot
# mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)